In [1]:
try:
    if __file__ != '__main__.py': 
        debug = False
except: 
    debug = True
    !jupyter nbconvert --to script __main__.ipynb
    !del app.py
    !move __main__.py app.py

[NbConvertApp] Converting notebook __main__.ipynb to script
[NbConvertApp] Writing 16214 bytes to __main__.py
/bin/bash: del: command not found
/bin/bash: move: command not found


In [2]:
import time , datetime
_ScriptStartAt = time.time()
now = lambda:datetime.datetime.now().strftime("%Y%m%d%H%M%S_%f")

_ScriptStartAt

1700607659.6891136

In [3]:
config = {
    "STORAGE":"."   # WHERE ALL HASH FILES GONNA BE SAVED
    # "STORAGE":"\\\\?\\Volume{17b24389-ad3f-48b8-b5bd-553b6a38e726}\\STORAGE", # WHERE ALL HASH FILES GONNA BE SAVED
    # "INDEX.db":"W:/STORAGE/index.sqlite3" # WHERE ALL HISTORY WILL BE REMEMBERED IN SHITSTORM SHITSHOW CRAP HAPPENS
} 

# Funções de Backup - Salvamento

In [4]:
def spawn2stdout(cmd):
    if type(cmd) == bytes:
        cmd = cmd.decode()
    
    import subprocess,sys
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout=bytes()
    stderr=bytes()

    while process.poll() == None or len(out) != 0:
        out = process.stdout.readline(1)
        stdout += out
        out = process.stderr.readline(1)
        stderr += out
    return stdout , stderr

In [6]:
def trycatch(func,errfunc):
    try:
        return func()
    except Exception as e:
        if errfunc:
            return errfunc(e)
        return e

In [5]:
import zlib

class crc32(object):
    name = 'crc32'
    digest_size = 4
    block_size = 1

    def __init__(self):
        self.__digest = 0

    def digest(self):
        return list((self.__digest).to_bytes(4, byteorder='big'))

    # def digest(self):
    #     # return '{:08x}'.format(self.__digest)
    #     return list(self.__digest.to_bytes(4, byteorder='big'))

    def update(self, arg):
        self.__digest = zlib.crc32(arg, self.__digest) & 0xffffffff

# Now you can define hashlib.crc32 = crc32
import hashlib
hashlib.crc32 = crc32

In [7]:
def hash_file(filename,tempDir=config["STORAGE"] + "\\temp\\"):
    import os, os.path as path
    assert path.isfile(filename) , "File not found for hash operation"

    class length():
        def __init__(self):
            self.size=0
        def update(self,data):
            self.size+=len(data)
        def digest(self):
            return list(self.size.to_bytes(8, byteorder='big'))

    class sumAll():
        def __init__(self):
            self.size=0
        def update(self,data):
            self.size+=sum(list(data))
        def digest(self):
            return list(self.size.to_bytes(10, byteorder='big'))

    class copyFile():
        def __init__(self):
            try:
                if tempDir is None: return
                import datetime
                import random
                # currentDT = datetime.datetime.now()
                # self.fname =  "f:\\temp\\" + currentDT.strftime("%Y%m%d%H%M%S_%f_")+str( random.randint(0,99999) ).zfill(5)+'.tmp'
                self.fname =  tempDir + datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f_")+str( random.randint(0,99999) ).zfill(5)+'.tmp'
                os.makedirs( os.path.dirname( self.fname ) ,exist_ok=True )
                self.fopen = open(self.fname , "wb" )
            except:
                self.fopen.close()
                raise Exception(self.fname," Path Cannot be written. ")
        def update(self,data):
            if tempDir is None: return
            try:
                self.fopen.write(data)
            except:
                self.fopen.close()
                raise
            None
        def digest(self):
            if tempDir is None: return
            self.fopen.close()
            return self.fname

    hashes = [ #### make a hash object

        # [ 'blake2b' , hashlib.blake2b() ],
        # [ 'blake2s' , hashlib.blake2s() ],
        # [ 'sha224' , hashlib.sha224() ],
        # [ 'sha384' , hashlib.sha384() ],
        # [ 'sha3_224' , hashlib.sha3_224() ],
        # [ 'sha3_256' , hashlib.sha3_256() ],
        # [ 'sha3_384' , hashlib.sha3_384() ],
        # [ 'sha3_512' , hashlib.sha3_512() ],
        # [ 'sha512' , hashlib.sha512() ],
        # [ 'shake_128' , hashlib.shake_128() ],
        # [ 'shake_256' , hashlib.shake_256() ],
        [ 'crc32' , hashlib.crc32() ] ,
        [ 'md5' , hashlib.md5() ] ,
        [ 'sha1' , hashlib.sha1() ] ,
        [ 'sha256' , hashlib.sha256() ] ,
        [ "sumAll" , sumAll() ] ,
        [ "length" , length() ] ,
        [ "copyFPath" , copyFile() ],

    ]

    with open(filename,'rb') as file:   ### open file for reading in binary mode
        chunk = 0
        while chunk != b'':
            chunk = file.read(1024*4 )  ### read file in chunks and update hash
            # h_sha256.update(chunk) ### old code. below there is a new way to do this
            for _,hash2 in hashes:
                hash2.update( chunk )

    hashResult = []
    for hashName , hashFunc in hashes:  ### return the hex digest of each hash engine
        tmp = hashFunc.digest()
        try:
            hashResult.append( [ hashName , "".join([ ("0"+hex(x)[2:])[-2:] for x in list( tmp ) ] ) ] ) ### convert to hex
        except:
            hashResult.append( [ hashName , tmp ] ) ### save as it is
    return hashResult

In [8]:
import re,os
def getAllDisksMount():
    # disksList = re.findall(r"(\\\\?\\.*\{[0-9a-f]{8}-[0-9a-f]{4}\-[0-9a-f]{4}\-[0-9a-f]{4}\-[0-9a-f]{12}\}\\)",os.popen("mountvol").read(),re.MULTILINE) ## "wmic VOLUME get DeviceID"
    disksList = re.findall(r"(\\\\?\\.*\{[0-9a-f]{8}-[0-9a-f]{4}\-[0-9a-f]{4}\-[0-9a-f]{4}\-[0-9a-f]{12}\}\\)",os.popen("wmic VOLUME get DeviceID").read(),re.MULTILINE)
    letters = [ chr(x) + ":\\" for x in range(65,90) if os.path.exists( chr(x) + ":\\" ) ]

    for vol , letter in re.findall(r"(\\\\?\\.*\{[0-9a-f]{8}-[0-9a-f]{4}\-[0-9a-f]{4}\-[0-9a-f]{4}\-[0-9a-f]{12}\}\\)\n\s+(.*)",os.popen("mountvol").read(),re.MULTILINE):
        try:
            if vol in disksList:
                letters.remove( letter[:2]+"\\" )
        except: None

    for letter in re.findall(r"\nOK\s+([a-zA-Z]:)\s+\\\\.+?\s+Microsoft Windows Network" , os.popen("net use").read() ):
        letters.remove(letter+"\\")

    return disksList + letters
getAllDisksMount()

/bin/sh: 1: wmic: not found
/bin/sh: 1: mountvol: not found
/bin/sh: 1: net: not found


[]

In [10]:
def listdir(path):
    try: tmp = [ os.path.join(path,x) for x in os.listdir(path) ] 
    except: tmp = []
    tmp.sort()
    return tmp
# listdir = lambda path:[ os.path.join(path,x) for x in os.listdir(path) ] 

def walkdir(path):
    paths = [ path ]
    while len(paths) > 0:

        path = paths.pop(0)

        if path.lower() == "SHA256_SHA1_MD5".lower():
            continue
        
        try:
            os.readlink(path)
            yield path
            continue
        except:
            None

        try:
            if os.path.isdir(path):
                if os.path.basename(path).lower() in [ path_x.lower() for path_x in ["DUMP","Sysmon",".ipynb_checkpoints","CacheManager","Steamlibrary","htmlcache","SteamApps","log","temp","tmp","STORAGE","AppCache","cacho","raw","morgue","Cache","cache2","Caches","Code Cache","CacheStorage","Code","D3DSCache","GPUCache","GrShaderCache","INetCache","jumpListCache","optimization_guide_hint_cache_store","ScriptCache","shader-cache","ShaderCache","shadercache","startupCache","$RECYCLE.BIN","System Volume Information","Temporary Internet Files"] ]:
                    continue
                paths = [ os.path.join(path,x) for x in os.listdir(path) ] + paths
                continue
            else:
                tmp = path.lower()
                if True in [ tmp.endswith( ext.lower() ) for ext in [ ".STORAGETHIS" , ".tmp" , ".temp" , ".log" , ".ztmp" , ".evt" , ".evtx" , ".dmp" , ".dump"  , ".etl" ] ]:
                    continue
                yield path
        except KeyboardInterrupt:
            raise
        except:
            None

def walkdirs(path):
    assert type(path) == list, "Expected 'list' "

    paths = [ walkdir(x) for x in path ]
    x = 0
    while True:
        try:
            yield next( paths[x] )
        except KeyboardInterrupt:
            raise
        except StopIteration:
            paths.remove( paths[x] )
            x+=-1
        except:
            if len( paths ) == 0:
                return
            paths.pop(x)
            x+=-1
        x+=1
        if x >= len( paths ):
            x=0

In [11]:
listaArquivos = set()

In [13]:
def __Worker_Backup():
    global status , listaArquivos
    status = '{"result":"Initiated"}'
    import random , datetime

    fNameBkp = "Mapping_" + str(_ScriptStartAt) + ".csv"
    with open( config["STORAGE"] + "\\"+ fNameBkp,"ab" ) as f_io:
        filehashed_Count=1
        while filehashed_Count !=0: ### in case no files are found, stop the script
            filehashed_Count=0
            try:
                for filePath in walkdirs( ["."] ):#getAllDisksMount() ):
                    sha256,sha1,md5,stat = None,None,None,None
                    try:
                        if "\\temp\\"   in  filePath :
                            continue
                        if "/temp/"     in  filePath :
                            continue
                        stat = os.stat( filePath )
                        fileStats = [ [ "n_fields" , stat.n_fields ] ,[ "n_sequence_fields" , stat.n_sequence_fields ] ,[ "n_unnamed_fields" , stat.n_unnamed_fields ] ,[ "st_atime" , stat.st_atime ] ,[ "st_atime_ns" , stat.st_atime_ns ] ,[ "st_ctime" , stat.st_ctime ] ,[ "st_ctime_ns" , stat.st_ctime_ns ] ,[ "st_dev" , stat.st_dev ] ,[ "st_file_attributes" , stat.st_file_attributes ] ,[ "st_gid" , stat.st_gid ] ,[ "st_ino" , stat.st_ino ] ,[ "st_mode" , stat.st_mode ] ,[ "st_mtime" , stat.st_mtime ] ,[ "st_mtime_ns" , stat.st_mtime_ns ] ,[ "st_nlink" , stat.st_nlink ] ,[ "st_reparse_tag" , stat.st_reparse_tag ] ,[ "st_size" , stat.st_size ] ,[ "st_uid" , stat.st_uid ] ]
                        fileSize = stat.st_size
                        if ( filePath + str(stat.st_ctime_ns) + str(stat.st_mtime_ns) ) in listaArquivos:
                            continue

                        ### All Green, lets hash the file, then copy it to the backup folder
                        
                        message = hash_file( filePath )# , tempDir="T:\\STORAGE\\TEMP\\" )
                        tmp = [ [ "filePath" , filePath ] ] + message + [ [ "scriptStart" , _ScriptStartAt ],[ "hashTimeAt" , time.time() ] ] + fileStats

                        f_io.write( "*".join( [ str( x[1] ) for x in tmp ] ).encode() )
                        f_io.write( "\r\n".encode() )

                        for varName , Val in message:
                            if varName == "md5": md5 = Val
                            if varName == "sha1": sha1 = Val
                            if varName == "sha256": sha256 = Val
                            if varName == "copyFPath": copyFPath = Val

                        temporarily = copyFPath
                        StoragePath = os.path.join( config["STORAGE"] , "SHA256_SHA1_MD5" , sha256[0:2] , sha256[2:4] , sha256 + "_" + sha1 + "_" + md5  )

                        os.makedirs( os.path.dirname( StoragePath ) , exist_ok=True )                        
                        trycatch( lambda: spawn2stdout( 'cmd /c move "'+temporarily+'" "'+StoragePath+'" ' ) )

                        try:    listaArquivos.add( ( filePath + str(stat.st_ctime_ns) + str(stat.st_mtime_ns) ) )
                        except: listaArquivos.append( ( filePath + str(stat.st_ctime_ns) + str(stat.st_mtime_ns) ) )


                        filehashed_Count+=1 ### tell the script that at least one file was hashed
                        status = '{"result":"Working"}'

                    except KeyboardInterrupt as Keyb:
                        raise
                    except Exception as Err:
                        if Err.args in [
                            ('File not found for hash operation',),
                            
                            (2 , 'The system cannot find the path specified'),
                            (2 , 'The system cannot find the file specified'),
                            (2 , 'The network path was not found'),
                            
                            (13, 'Permission denied'),
                            (13, 'Access is denied'),
                            (13, 'The device is not ready'),
                            
                            (22, 'Invalid argument'),
                            (22, 'A device which does not exist was specified'),
                            (22, 'The file cannot be accessed by the system'),
                            (22, 'The symbolic link cannot be followed because its type is disabled'),
                            (22, 'An unexpected network error occurred'),
                            (22, 'The semaphore timeout period has expired'),                            
                        ]:
                            continue

            except KeyboardInterrupt as Keyb:
                status = '{"result":"ERROR_KeyboardInterrupt"}'
                raise
            except StopIteration:
                status = '{"result":"ERROR_StopIteration"}'
                continue
            except Exception as Err:
                status = '{"result":"ERROR_Exception"}'
                None
    status = '{"result":"Done"}'

In [14]:
thread1 = None
def __Worker_Backup_START():
    global thread1
    try:
        if not thread1.is_alive():
            raise 0
        # print(" Thread is \033[96m ALREADY \033[1;0m Running")
        return '{"result":"RunningAlready"}'
    except:
        from threading import Thread
        thread1 = Thread( target=__Worker_Backup , args=() )
        thread1.start()
        # print(" Thread has \033[95m STARTED \033[1;0m Running")
        return '{"result":"OK"}'

# Funções de Backup - Carregamento

In [12]:
try:
    with open("Mapping_1699371619.749871_20231107124020_699131_59895.csv","r") as file:
        Dados = file.readline().split("*")
        tmp = ( Dados[0] + Dados[16] + Dados[23] )
        try:    listaArquivos.   add( tmp )
        except: listaArquivos.append( tmp )
except:
    None

In [15]:
if debug:
    %pip install flask


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import os

# Motor do site - Flask

In [17]:
from flask import Flask , Response , redirect
if debug:
    app = Flask("")#__name__)
else:
    app = Flask(__name__)

# Main Page
@app.route('/')
def hello():
    return redirect("/index.html", code=302)

# Generic Resource
get_resource = lambda *argp:"<html><head/><body/></html>"
@app.route('/<path:path>')
def __get_resource( path ):  # pragma: no cover
    global get_resource
    return get_resource( path )

# Backup Thread Status
status = '{"result":"NotStarted"}'
@app.route('/api/v0/status')
def __status():  # pragma: no cover
    global status
    return status

# Trigger Backup Thread
@app.route('/api/v0/startBackup')
def __Worker_Backup_Status( ):  # pragma: no cover
    global __Worker_Backup_START
    return __Worker_Backup_START()

# FileTreeExplorer
get_path = lambda *argp:"<html><head/><body><table/></body></html>"
@app.route('/api/v0/getPath/<path:path>')
def __get__path( path ):  # pragma: no cover
    global get_path
    return get_path( path )

# get the file that was backuped-up in one of the previously backup threads
get_filePath = lambda *argp:"<html><head/><body><table/></body></html>"
@app.route('/api/v0/getPath/<path:path>')
def __get__filePath( path ):  # pragma: no cover
    global get_filePath
    return get_filePath( path )

# app.debug = True
__main__ = lambda: app.run( host='127.0.0.1' , port=18080 , debug=not debug )

if debug:
    from threading import Thread
    Thread( target=__main__, args=() ).start()

 * Serving Flask app ''
 * Debug mode: off


In [24]:
def get_resource( path ):  # pragma: no cover
    print("###" , path )
    mimetypes = {
        ".css": "text/css",
        ".html": "text/html",
        ".js": "application/javascript",
    }
    if os.path.exists( path ):
        with open( path ) as f_io:
            return Response( f_io.read() , mimetype=mimetypes.get( path.split(".")[-1] , "text/html") )
    else:
        return Response( status=404 )

127.0.0.1 - - [21/Nov/2023 23:15:27] "GET /index.html HTTP/1.1" 200 -


### index.html


127.0.0.1 - - [21/Nov/2023 23:15:28] "GET /public/linkedin.svg HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 23:15:28] "GET /public/instagram.svg HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 23:15:28] "GET /public/twitter.svg HTTP/1.1" 200 -


### public/linkedin.svg
### public/instagram.svg
### public/twitter.svg


In [ ]:
import sqlite3
db = sqlite3.connect(':memory:')
# db = sqlite3.connect( config["STORAGE"] + r"\STORAGE.db" )

In [ ]:
def __get_path_Backups__():
    get_path_Status = 1
    def loopableThingy():
        

    get_path_Status = 2
    return None

In [ ]:
import pandas as pd

In [26]:
get_path_Status = 0
def get_path( path ):  # pragma: no cover
    if get_path_Status = 0:
        from threading import Thread
        Thread( target=__get_path_Backups__ ).start()
        return '<table><tr><td>Loading</td></tr></table><script>setTimeout( function(){ document.getElementById("buttonUpdate"); } , 1000 )</script>'

    else:
        body = ""
        if get_path_Status = 1:
            body += '<h1>A Listagem de arquivos esta incompleta.</h1><p>ainda esta sendo carregada em memoria</p>'
        body += '<table><tr><td></td></tr></table>'
        body += '<table><tr><td></td></tr></table>'

        return body
        
    return "NOT IMPLEMENTED"

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3356292270.py, line 3)

In [19]:
# @app.root()
# def __HTMLRoot__(ur):
    # with open("index.html") as f_io:
        # return f_io.read()


Press CTRL+C to quit

In [20]:
def HTMLRoot():
    with open("index.html") as f_io:
        return f_io.read()

In [21]:
HTMLRoot()

'<html lang="en" class="sr">\n    <head>\n        <title> SeiLá Backups </title>\n        <meta charset="utf-8">\n        <script src="https://unpkg.com/htmx.org@1.9.9"/>\n        <style>\n            html {  line-height: 1.15; }\n            body {  margin: 0; }\n            * {  box-sizing: border-box;  border-width: 0;  border-style: solid; }\n            p,li,ul,pre,div,h1,h2,h3,h4,h5,h6,figure,blockquote,figcaption {  margin: 0;  padding: 0; }\n            #button {  background-color: transparent; }\n            button,input,optgroup,select,textarea {  font-family: inherit;  font-size: 100%;  line-height: 1.15;  margin: 0; }\n            button,select {  text-transform: none; }\n            button,[type="button"],[type="reset"],[type="submit"] {  -webkit-appearance: button; }\n            a {  color: inherit;  text-decoration: inherit; }\n            input {  padding: 2px 4px; }\n            img {  display: block; }\n        </style>\n    </head>\n    <body style="height: 100%;wid

127.0.0.1 - - [21/Nov/2023 23:01:16] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [21/Nov/2023 23:01:16] "GET /index.html HTTP/1.1" 200 -


### index.html


127.0.0.1 - - [21/Nov/2023 23:01:22] "GET /index.html HTTP/1.1" 200 -


### index.html


127.0.0.1 - - [21/Nov/2023 23:02:00] "GET /index.html HTTP/1.1" 200 -


### index.html


127.0.0.1 - - [21/Nov/2023 23:02:01] "GET /public/instagram.svg HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 23:02:01] "GET /public/linkedin.svg HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2023 23:02:01] "GET /public/twitter.svg HTTP/1.1" 200 -


### public/linkedin.svg
### public/instagram.svg
### public/twitter.svg
